In [1]:
from copy import deepcopy

import torch as t
from torch.optim import Adam

from modeling.chessformer import ChessFormerA2C
from reinforcement.actor_critic.a2c import A2CTrainer
from reinforcement.players import A2CModelPlayer

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MAX_BATCH_SIZE = 128

In [3]:
model = ChessFormerA2C(nb_encoder_layers=6, nb_decoder_layers_policy=3, embedding_dim=512,
                       bottleneck_hidden_dim=2048, dim_per_head=64, nb_head=8).to('cuda')

model.load_state_dict(t.load('runs/a2c_test_lr5e-6_history_len_1_vs_opponent_2/model.pt'))

optimizer = Adam(model.parameters(), lr=5e-6)

In [4]:
for i in range(3, 21):
    
    competitor = A2CModelPlayer(deepcopy(model), model_device='cuda')
    
    experiment_name = f'a2c_test_lr5e-6_history_len_1_vs_opponent_{i}'
    
    trainer = A2CTrainer(model,
                         optimizer,
                         competitor,
                         MAX_BATCH_SIZE,
                         experiment_name,
                         model_device='cuda',
                         history_len=1)
    
    trainer.train(50_000)
    
    t.save(model.state_dict(), f'runs/{experiment_name}/model.pt')

2023-12-08 12:40:27.914084: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-08 12:40:28.695852: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-12-08 12:40:28.695898: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-12-08 12:40:28.695901: W 